In [51]:
import xarray as xr
import numpy as np

Conservative Coarseninf via Area Weighted Mean without paying any heed to the target grid here

In [ ]:
def conservative_area_weighted_coarsen(pr, lat2d, lon2d, nlat_block, nlon_block):
    """
    Conservatively coarsen a 2D or 3D field using area-weighted averaging on a 2D curvilinear grid.

    Parameters:
        pr : xr.DataArray (time, y, x) or (y, x)
        lat2d, lon2d : 2D latitude and longitude arrays (same shape as pr)
        nlat_block, nlon_block : int, coarsening factor in lat/lon directions

    Returns:
        xr.DataArray: coarsened field
    """
    R = 6371000  # Earth radius in meters

    # Handle optional time dimension
    has_time = 'time' in pr.dims
    if not has_time:
        pr = pr.expand_dims('time')

    nt, ny, nx = pr.shape

    # Trim to fit exact block sizes
    ny_trim = (ny // nlat_block) * nlat_block
    nx_trim = (nx // nlon_block) * nlon_block

    pr = pr[:, :ny_trim, :nx_trim]
    lat2d = lat2d[:ny_trim, :nx_trim]
    lon2d = lon2d[:ny_trim, :nx_trim]

    # Compute approximate area per grid cell (cos(lat) scaling)
    dlat = np.deg2rad(np.diff(lat2d[:, 0]).mean())
    dlon = np.deg2rad(np.diff(lon2d[0, :]).mean())
    area = (R ** 2) * dlat * dlon * np.cos(np.deg2rad(lat2d))

    # Reshape into blocks
    pr_vals = pr.values
    area_vals = area

    pr_blocks = pr_vals.reshape(
        nt, ny_trim // nlat_block, nlat_block,
        nx_trim // nlon_block, nlon_block
    )

    area_blocks = area_vals.reshape(
        ny_trim // nlat_block, nlat_block,
        nx_trim // nlon_block, nlon_block
    )

    # Compute area-weighted average
    weighted_sum = (pr_blocks * area_blocks).sum(axis=(2, 4))
    total_area = area_blocks.sum(axis=(1, 3))
    pr_coarse = weighted_sum / total_area

    # Coarsen lat/lon for coordinates
    lat_blocks = lat2d.reshape(ny_trim // nlat_block, nlat_block,
                               nx_trim // nlon_block, nlon_block)
    lon_blocks = lon2d.reshape(ny_trim // nlat_block, nlat_block,
                               nx_trim // nlon_block, nlon_block)

    lat_coarse = lat_blocks.mean(axis=(1, 3))
    lon_coarse = lon_blocks.mean(axis=(1, 3))

    coords = {
        'lat': (['y', 'x'], lat_coarse),
        'lon': (['y', 'x'], lon_coarse),
    }

    if has_time:
        coords['time'] = pr['time']
        dims = ('time', 'y', 'x')
    else:
        pr_coarse = pr_coarse.squeeze()
        dims = ('y', 'x')

    return xr.DataArray(pr_coarse, coords=coords, dims=dims, name=pr.name)


In [46]:
block_size = 11  # 0.11 degrees from ~1km
pr_coarse2 = conservative_area_weighted_coarsen(
    ds['RhiresD'],
    ds['lat'].values,
    ds['lon'].values,
    block_size, block_size
)


In [50]:
pr_coarse2.to_netcdf("RhiresD_coarsened_typ2_011deg.nc")